In [ ]:
using Gmsh: gmsh
using Gridap
using GridapGmsh
using Gridap.TensorValues
using LineSearches: BackTracking
using Gridap.Arrays
using Gridap.ReferenceFEs
using Plots
using LinearAlgebra

In [ ]:
I2 = SymTensorValue{2,Float64}(1.0 ,0.0 ,1.0)
I4 = I2⊗I2
I4_sym = one(SymFourthOrderTensorValue{2,Float64})
I4_vol = (1.0/2)*I4
I4_dev = I4_sym  - I4_vol

In [ ]:
const ls = 0.85

In [ ]:
const L = 100
const CH = 25         #Crack height
const CW = 0.01*CH    #Crack Width
const hf = 0.5/3.2 #CH/100     #Mesh size parameter
const h = 15*hf       #Mesh size parameter

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)

p1 = gmsh.model.geo.addPoint(0.0, 0.0, 0.0, h)
p2 = gmsh.model.geo.addPoint(L, 0, 0.0, h)
p3 = gmsh.model.geo.addPoint(L, 0.5*(L-CW), 0.0, h)
p4 = gmsh.model.geo.addPoint(L-CH, 0.5*(L-CW), 0.0, h)
p5 = gmsh.model.geo.addPoint(L-CH, 0.5*(L+CW), 0.0, h)
p6 = gmsh.model.geo.addPoint(L, 0.5*(L+CW), 0.0, h)
p7 = gmsh.model.geo.addPoint(L, L, 0.0, h)
p8 = gmsh.model.geo.addPoint(0.0, L, 0.0, h)
p9 = gmsh.model.geo.addPoint(0.0, 0.5*(L+CW), 0.0, h)
p10 = gmsh.model.geo.addPoint(CH, 0.5*(L+CW), 0.0, h)
p11 = gmsh.model.geo.addPoint(CH, 0.5*(L-CW), 0.0, h)
p12 = gmsh.model.geo.addPoint(0.0, 0.5*(L-CW), 0.0, h)

l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p5)
l5 = gmsh.model.geo.addLine(p5, p6)
l6 = gmsh.model.geo.addLine(p6, p7)
l7 = gmsh.model.geo.addLine(p7, p8)
l8 = gmsh.model.geo.addLine(p8, p9)
l9 = gmsh.model.geo.addLine(p9, p10)
l10 = gmsh.model.geo.addLine(p10, p11)
l11 = gmsh.model.geo.addLine(p11, p12)
l12 = gmsh.model.geo.addLine(p12, p1)

cl1 = gmsh.model.geo.addCurveLoop([l1,l2,l3,l4,l5,l6,l7,l8,l9,l10,l11,l12])
ps1 = gmsh.model.geo.addPlaneSurface([cl1])

pg1 = gmsh.model.addPhysicalGroup(2, [ps1])

pg2 = gmsh.model.addPhysicalGroup(1, [l1])
pg3 = gmsh.model.addPhysicalGroup(1, [l7])

gmsh.model.setPhysicalName(2, pg1, "Domain")
gmsh.model.setPhysicalName(1, pg2, "DirichletBottom")
gmsh.model.setPhysicalName(1, pg3, "LoadLine")

gmsh.model.mesh.field.add("Box", 11)
gmsh.model.mesh.field.setNumber(11, "VIn", hf)
gmsh.model.mesh.field.setNumber(11, "VOut", h)
gmsh.model.mesh.field.setNumber(11, "XMin", 0.5*(L-2.25*CH))
gmsh.model.mesh.field.setNumber(11, "XMax", 0.5*(L+2.25*CH))
gmsh.model.mesh.field.setNumber(11, "YMin", 0.5*(L-1.75*CH))
gmsh.model.mesh.field.setNumber(11, "YMax", 0.5*(L+1.75*CH))

gmsh.model.mesh.field.setAsBackgroundMesh(11)

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("DoubleEdgeNotch.msh")
gmsh.finalize()

In [ ]:
model = GmshDiscreteModel("DoubleEdgeNotch.msh")
writevtk(model,"DoubleEdgeNotch")

In [ ]:
using Gridap.Geometry
labels = get_face_labeling(model)
dimension = 2
mat_tags = get_face_tag(labels,dimension)

In [ ]:
const E = 30e3
const ν = 0.2
const G = E/(2*(1+ν))
const l = 15.0
const N = 0.5
const Gc = 0.1

const η = 1e-15

In [ ]:
const λ = 2*G*ν/(1 -2*ν)
const κ = 2*G*N^2/(1-N^2)
const μ = G*(1-2*(N^2))/(1-N^2)
const γ = 4*G*l^2

In [ ]:
const σc = 7.745  # since ψ_Crit = 0.001 N/mm^2
const m = 3*E*Gc/(4*ls*(σc^2))
const ψ_Crit = σc^2 / (2*E) #ψ_Crit = 0.001 N/mm^2
const kf = 0.5
const αT = ψ_Crit/kf

In [ ]:
path = "./DENT_lb15_N05_ls$ls/"                                # Output path
mkpath(path) 
cd("DENT_lb15_N05_ls$ls/")

In [ ]:
function degDer(ϕ)
    g = (m*ϕ)/(0.25*m^2*ϕ^4 - 0.5*m^2*ϕ^2 + 0.25*m^2 - m*ϕ^4 + m*ϕ^2 + ϕ^4) 
    return g
end

In [ ]:
function σ_Bmod(ε, ε_in, s_in)
    gs = (s_in^2) / (s_in^2 + m*(1-s_in)*(0.5+0.5*s_in))
    if tr(ε_in) >= 0
        σM = (gs)*((λ + μ + (κ/2))*tr(ε)*one(ε) + (2*μ + κ)*(I4_dev ⊙ ε))
    else
        σM = (gs)*((2*μ + κ)*(I4_dev ⊙ ε)) + (λ + μ + (κ/2))*tr(ε)*one(ε)
    end
    return σM
end

In [ ]:
E_Matrx = TensorValue(0,1,-1,0)

In [ ]:
function ε_Skw(∇,θ)
    ∇ᵀ = transpose(∇)
    w = (0.5*(∇ᵀ - ∇)) - (E_Matrx*θ)
    return w
end

In [ ]:
function σ_Cmod(ϵ_skew, s_in)
    gs = (s_in^2) / (s_in^2 + m*(1-s_in)*(0.5+0.5*s_in))
    σM = (gs)*(κ*ϵ_skew)
    return σM
end

In [ ]:
function M_mod(∇, s_in)
    gs = (s_in^2) / (s_in^2 + m*(1-s_in)*(0.5+0.5*s_in))
    M = (gs)*(γ*∇)
    return M
end

In [ ]:
function ψPos(ε_in,∇_uh,θ, ∇_th,Fdg)
    ∇_uhᵀ = transpose(∇_uh)
    ϵ_skew = 0.5*(∇_uhᵀ - ∇_uh) - E_Matrx*θ
    if tr(ε_in) >= 0
        ψPos = (0.5*((λ + μ + (κ/2))*(tr(ε_in))^2 + (2*μ+κ)*((I4_dev ⊙ ε_in) ⊙ (I4_dev ⊙ ε_in))) + 0.5*(κ*(ϵ_skew ⊙ ϵ_skew)) + 0.5*γ*(∇_th ⋅ ∇_th))/Fdg
    else
        ψPos = (0.5*((2*μ+κ)*((I4_dev ⊙ ε_in) ⊙ (I4_dev ⊙ ε_in))) + 0.5*(κ*(ϵ_skew ⊙ ϵ_skew)) + 0.5*γ*(∇_th ⋅ ∇_th))/Fdg
    end
    return ψPos 
end

function ψPosOrg(ε_in,∇_uh,θ, ∇_th)
    ∇_uhᵀ = transpose(∇_uh)
    ϵ_skew = 0.5*(∇_uhᵀ - ∇_uh) - E_Matrx*θ
    if tr(ε_in) >= 0
        ψPos = 0.5*((λ + μ + (κ/2))*(tr(ε_in))^2 + (2*μ+κ)*((I4_dev ⊙ ε_in) ⊙ (I4_dev ⊙ ε_in))) + 0.5*(κ*(ϵ_skew ⊙ ϵ_skew)) + 0.5*γ*(∇_th ⋅ ∇_th)
    else
        ψPos = 0.5*((2*μ+κ)*((I4_dev ⊙ ε_in) ⊙ (I4_dev ⊙ ε_in))) + 0.5*(κ*(ϵ_skew ⊙ ϵ_skew)) + 0.5*γ*(∇_th ⋅ ∇_th)
    end
    return ψPos 
end

In [ ]:
function new_EnergyState(ψPlusPrev_in,ψhPos_in)
    ψPlus_in = ψhPos_in
    if ψPlus_in >= ψPlusPrev_in
        ψPlus_out = ψPlus_in
    else
        ψPlus_out = ψPlusPrev_in
    end
    true,ψPlus_out
end

In [ ]:
function project(q,model,dΩ,order)
  reffe = ReferenceFE(lagrangian,Float64,order)
  V = FESpace(model,reffe,conformity=:L2)
  a(u,v) = ∫( u*v )*dΩ
  l(v) = ∫( v*q )*dΩ
  op = AffineFEOperator(a,l,V,V)
  qh = solve(op)
  qh
end

In [ ]:
function FatigueDegrad(αb)
    fdg = @. ifelse(αb >= αT , ((2*αT)/(αb + αT))*((2*αT)/(αb + αT)), 1.0)
    return fdg
end

In [ ]:
function FatiguehistoryVariable(ψhPos_in,αPrev,αbPrev)
    α = ψhPos_in #(s_in ^2 + η)*ψhPos_in
    if α >= αPrev
       αb = αbPrev + abs(α-αPrev)
    else
       αb = αbPrev
    end
return αb
end

In [ ]:
function αNC(s_in,ψhPos_in)
    gs = (s_in^2) / (s_in^2 + m*(1-s_in)*(0.5+0.5*s_in))
    α = gs * ψhPos_in
return α
end

In [ ]:
order = 1
degree = 2*order

In [ ]:
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

In [ ]:
sId = CellState(1.0,dΩ)
shId = project(sId,model,dΩ,order)

In [ ]:
LoadTagId = get_tag_from_name(labels,"LoadLine")
Γ_Load = BoundaryTriangulation(model,tags = LoadTagId)
dΓ_Load = Measure(Γ_Load,degree)
n_Γ_Load = get_normal_vector(Γ_Load)

In [ ]:
reffe_PF = ReferenceFE(lagrangian,Float64,order)
V0_PF = TestFESpace(model,reffe_PF;conformity=:H1)
sh = zero(V0_PF)

In [ ]:
reffe_theta = ReferenceFE(lagrangian,Float64,order)
V0_theta  = TestFESpace(model,reffe_theta;
  conformity=:H1,dirichlet_tags=["DirichletBottom"])
θh = zero(V0_theta)

In [ ]:
reffe_Disp = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
        V0_Disp = TestFESpace(model,reffe_Disp;
          conformity=:H1,
          dirichlet_tags=["DirichletBottom","LoadLine"],
          dirichlet_masks=[(true,true), (true,true)])
uh = zero(V0_Disp)

In [ ]:
V0 = MultiFieldFESpace([V0_Disp,V0_theta])

In [ ]:
function crack_tip_tracker(sh,tol)
    damagetracker = Float64[]
    sVec = Float64[]
    coords = get_node_coordinates(Ω)
    
    sVec = get_free_dof_values(sh)
    
    for i in 1:length(sVec)
        if sVec[i] <= tol
            push!(damagetracker,i)
        end
    end
   damagetracker = round.(Int,damagetracker)
    xloc = Float64[]
    yloc = Float64[]
    if damagetracker!= Float64[]
        for i in 1:length(damagetracker)
            push!(xloc,coords[damagetracker[i]][1])
            push!(yloc,coords[damagetracker[i]][2])
        end
        Locx, index= findmax(xloc)
        Locy = yloc[index]
        return (Locx,Locy)
    else 
       return "No damage"
    end
end

In [ ]:
nls = NLSolver(
  show_trace=true,
  method=:newton,
  linesearch=BackTracking(), iterations = 10)
solver = FESolver(nls)

In [ ]:
function stepPhaseField(x0,vApp,cache,ψPlusPrev_in)
     U_PF = TrialFESpace(V0_PF)
     res(s,ϕ) = ∫( (3/4)*Gc*ls*∇(ϕ)⋅ ∇(s) + (degDer∘(s))*ψPlusPrev_in*ϕ - ((3/8)*Gc/ls)*ϕ)*dΩ
     op = FEOperator(res,U_PF,V0_PF)
     sh_out = FEFunction(U_PF,x0)
     sh_out, cache = solve!(sh_out,solver,op,cache)
  return sh_out, get_free_dof_values(sh_out), cache
end

In [ ]:
function Skw(u,θ)
    ∇ᵀ = transpose(∇(u))
    w = (0.5*(∇ᵀ) - (E_Matrx*θ))
    return w
end

In [ ]:
function   stepDisp(uh_in,θ_in,sh_in,uApp)
        uApp1(x) = VectorValue(0.0,0.0)
        uApp2(x) = VectorValue(uApp,uApp)
        U_Disp = TrialFESpace(V0_Disp,[uApp1,uApp2])
        
        θApp1(x) = 0.0
        U_theta = TrialFESpace(V0_theta,[θApp1])
        U = MultiFieldFESpace([U_Disp,U_theta ])

        a((u,θ),(w,v))  = ∫( ((Skw(w,v)) ⊙ (σ_Bmod∘(ε(u),ε(uh_in),sh_in)) ) + ((Skw(w,v)) ⊙ (σ_Cmod∘(ε_Skw∘(∇(u),θ), sh_in)) ) + ((∇(v))⋅ (M_mod∘(∇(θ),sh_in))) - (v*((E_Matrx) ⊙ (σ_Cmod∘(ε_Skw∘(∇(u),θ),sh_in))) ))*dΩ
        b((w,v))= 0
        op_Disp = AffineFEOperator(a,b,U,V0)
        uh_out = solve(op_Disp)
        uh_out,phih_out = uh_out
    return uh_out, phih_out
end

In [ ]:
vApp = 0
const innerMax = 10
const tol_lim = 1e-10
count = 0

const T = 1
const n = 200
const Tₚ = T/n
const n1 = 16 # loadsteps
const delv = 0.005 #*cos(pi/4)
const tp = Tₚ/n1
t = 0.0
cycle = 0
const dc = 1/n1

LoadVert = Float64[]
LoadHort = Float64[]
Displacement = Float64[]
time = Float64[]
AppDisplacement = Float64[]
αEnergy = Float64[]
αDegrad = Float64[]
NoofCycles = Float64[]
Xloccrack = Float64[]
Yloccrack = Float64[]
FracEnergy = Float64[]


push!(Xloccrack,(L/2))
push!(Yloccrack,0.5*L)
push!(LoadVert, 0.0)
push!(LoadHort, 0.0)
push!(Displacement, 0.0)

push!(time, t)
push!(AppDisplacement, vApp)
push!(NoofCycles, cycle)

push!(αEnergy, 0.0)
push!(αDegrad, 1.0)
push!(FracEnergy, 0.0)


sPrev = CellState(1.0,dΩ)
sh = project(sPrev ,model ,dΩ,order)
x0_PF = ones(Float64,num_free_dofs(V0_PF))
cache_1 = nothing

αPrev = CellState(0.0,dΩ)
αbPrev = CellState(0.0,dΩ)

αhPlusPrev = project(αPrev,model,dΩ,order)
αbhPlusPrev = project(αbPrev,model,dΩ,order)

FdhPrev = FatigueDegrad∘(αbhPlusPrev)

ψPlusPrev = CellState(ψ_Crit,dΩ)

while t .< T 
        
    t = t + tp
    vAppC = abs(4*delv/Tₚ * abs((((t-Tₚ/4)%Tₚ)+Tₚ)%Tₚ - Tₚ/2) - delv)
    vApp = round(vAppC,digits = 6)
    count = count .+ 1
    cycle = cycle + dc
    FdhPrev = FatigueDegrad∘(αbhPlusPrev)
   
    print("\n Entering displacemtent step$count :", float(vApp))
    
   for inner = 1:innerMax
        
        ψhPlusPrev = project(ψPlusPrev,model,dΩ,order)
        
        RelErr = abs(sum(∫( (3/4)*Gc*ls*∇(sh)⋅ ∇(sh) + (degDer∘(sh))*ψhPlusPrev*sh)*dΩ-∫( ((3/8)*Gc/ls)*sh)*dΩ))/abs(sum(∫( ((3/8)*Gc/ls)*sh)*dΩ))
        print("\n Relative error = ",float(RelErr))
        
        sh,x0_PF,cache_1 = stepPhaseField(x0_PF,vApp,cache_1,ψhPlusPrev) 
        uh,θh = stepDisp(uh,θh,sh,vApp)
        
        ψhPos_in = ψPos∘(ε(uh),∇(uh),θh,∇(θh),FdhPrev)      
        
        update_state!(new_EnergyState,ψPlusPrev,ψhPos_in)
        
        if RelErr < 1e-8
            break 
        end
    end
    
    if  crack_tip_tracker(sh,tol_lim) =="No damage"
        push!(Xloccrack,0.5*L)
        push!(Yloccrack,0.5*L)
    else
        (X_t, Y_t) = crack_tip_tracker(sh,tol_lim)
        push!(Xloccrack,X_t)
        push!(Yloccrack,Y_t)
    end
    
    ψhPosOrg_in = ψPosOrg∘(ε(uh),∇(uh),θh,∇(θh))
    αPrev = αNC∘(sh,ψhPosOrg_in)
    αbPrev = FatiguehistoryVariable∘(αPrev,αhPlusPrev,αbhPlusPrev)
    
    αhPlusPrev = project(αPrev,model,dΩ,order)
    αbhPlusPrev = project(αbPrev,model,dΩ,order)
    
    FdhPrev = FatigueDegrad∘(αbhPlusPrev)
    
    αbPrevVal = evaluate(αbPrev,VectorValue(0.5*L, 2*CH))
    Fdegrad = evaluate(FdhPrev,VectorValue(0.5*L, 2*CH))
    FracEnergyExp = sum(∫((3/8)*((1-sh)/ls + ls*(∇(sh) ⋅ ∇(sh))))*dΩ)
    
    push!(time, t)
    push!(AppDisplacement, vApp)
    push!(NoofCycles, cycle)
    
    push!(αEnergy, αbPrevVal)
    push!(αDegrad, Fdegrad)
    push!(FracEnergy, FracEnergyExp)
    
    Node_Force = sum(∫( n_Γ_Load ⋅ (σ_Bmod∘(ε(uh),ε(uh),sh)) ) *dΓ_Load + ∫( n_Γ_Load ⋅ (σ_Cmod∘(ε_Skw∘(∇(uh),θh),sh) ) )  *dΓ_Load)
    
    push!(LoadHort, Node_Force[1])
    push!(LoadVert, Node_Force[2])

    push!(Displacement, vApp)
    if mod(count,10*n1) == 0
         writevtk(Ω,"results_PhaseField_Mode1_N0.5lb15.0_$count",cellfields=
        ["uh"=>uh,"s"=>sh ,"θ" => θh, "epsi"=>ε(uh)])
    end
end

In [ ]:
writevtk(Ω,"results__PhaseField_Mode1_N0.5l0.10_$count",cellfields=
        ["uh"=>uh,"s"=>sh ,"θ" => θh, "epsi"=>ε(uh)])

In [ ]:
plot(Displacement,Load*1e-3)

In [ ]:
plot(NoofCycles,Xloccrack)

In [ ]:
parent_path = "../PlottingFiles/"     # Go up one level and define the new folder name
mkpath(parent_path)

In [ ]:
cd("../PlottingFiles")

In [ ]:
using DelimitedFiles

In [ ]:
NoofCyclesCSV = writedlm("kf05_Step16NoofCyclesPhlsfullVolDevlb15_N0.5_ls$ls.csv",  NoofCycles, ',')
FracEnergyCSV = writedlm("kf05_Step16FracEnergyPhlsfullVolDevlb15_N0.5_ls$ls.csv",  FracEnergy, ',')
LoadHortEnergyCSV = writedlm("kf05_Step16LoadHortPhlsfullVolDevlb15_N0.5_ls$ls.csv",  LoadHort, ',')
LoadVertEnergyCSV = writedlm("kf05_Step16LoadVertPhlsfullVolDevlb15_N0.5_ls$ls.csv",  LoadVert, ',')

In [ ]:
XloccrackCSV = writedlm("kf05_Step16XloccrackPhlsfullVolDevlb15_N0.5_ls$ls.csv",  Xloccrack, ',')
FdegradCSV = writedlm("kf05_Step16FdegradPhlsfullVolDevlb15_N0.5_ls$ls.csv",  αDegrad, ',')
αEnergyCSV = writedlm("kf05_Step16AlphaEnergyPhlsfullVolDevlb15_N0.5_ls$ls.csv",  αEnergy, ',')